## Ejercicios de pair programming 12 enero : ETL 3.2

Igual que en el ejercicio de pair programming de ETL Transformación II, tendréis que crear una clase que nos permita cargar los datos en SQL I.

In [17]:
#importamos las librerias
import pandas as pd
import mysql.connector

In [18]:
#abrimos el archivo de tiburones limpio
df_tiburon = pd.read_csv('../datos/tiburon8.csv', index_col = 0)

#abrimos el archivo del clima
df_clima = pd.read_csv('../datos/meteo_tiburon.csv', index_col= 0)



In [19]:
df_tiburon.head(2)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
1,2018.06.03.a,2018,Unprovoked,brazil,Swimming,FATAL,Jun,Y,M,Tiger,18.0


In [20]:
df_clima.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,pais
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 3}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 210, 'speed':...",12,15,4,1026,none,0,0,270,2,USA
1,6,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 345, 'speed':...",12,15,3,1027,none,0,0,15,2,USA


In [21]:
class Bbdd:

    def __init__ (self, nombre_bbdd, contraseña):
        self.nombre_bbdd = nombre_bbdd
        self.contraseña = contraseña


    def crear_bbdd(self):

        mydb = mysql.connector.connect(
        host="localhost",
        user="root",
        password=f"{self.contraseña}"
        )
        print("Conexión realizada con éxito")
        
        mycursor = mydb.cursor()

        try:
            mycursor.execute(f"CREATE DATABASE IF NOT EXISTS {self.nombre_bbdd};")
            print(mycursor)
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)



    #creamos una funcion para crear tabla nueva
    def crear_insertar_tabla(self, query):
        self.query = query
        
        # nos conectamsos con el servidor usando el conector de sql
        cnx = mysql.connector.connect(user='root', password=f"{self.contraseña}",
                                        host='127.0.0.1', database=f"{self.nombre_bbdd}")
        # iniciamos el cursor
        mycursor = cnx.cursor()
        
        # intentamos hacer la query
        try: 
            mycursor.execute(self.query)
            cnx.commit() 
        # en caso de que podamos ejecutar la query devuelvenos un error para saber en que nos estamos equivocando
        except mysql.connector.Error as err:
            print(err)
            print("Error Code:", err.errno)
            print("SQLSTATE", err.sqlstate)
            print("Message", err.msg)






In [22]:
mi_bbdd = Bbdd("tiburones", "AlumnaAdalab")

In [23]:
paso = mi_bbdd.crear_bbdd()

Conexión realizada con éxito
MySQLCursor: CREATE DATABASE IF NOT EXISTS tiburones;


In [24]:
#hacemos la query para la tabla ataques
tabla_ataques = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`ataques` (
  `case_number` VARCHAR(20) NOT NULL ,
  `year`INT NOT NULL,
  `type` VARCHAR(50) NOT NULL,
  `country` VARCHAR(50),
  `activity` VARCHAR (200),
  `injury` TEXT,
  `mes` VARCHAR (10),
  `deceso` VARCHAR(20),
  `genero` VARCHAR (20),
  `especies` VARCHAR (50),
  `edad` FLOAT,
  PRIMARY KEY (`case_number`))
ENGINE = InnoDB;
'''

In [25]:
tabla1 = mi_bbdd.crear_insertar_tabla(tabla_ataques)

In [26]:
#hacemos la query para la tabla clima

tabla_clima = '''
CREATE TABLE IF NOT EXISTS `tiburones`.`clima` (
  `index` INT NOT NULL AUTO_INCREMENT,
  `timepoint` INT,
  `cloudcover` INT, 
  `highcloud` INT, 
  `midcloud` INT, 
  `lowcloud` INT,
  `rh_profile` TEXT, 
  `wind_profile` TEXT, 
  `temp2m` INT, 
  `lifted_index` INT, 
  `rh2m` INT,
  `msl_pressure` INT, 
  `prec_type` VARCHAR (10), 
  `prec_amount` INT, 
  `snow_depth` INT,
  `wind10m_direction` INT, 
  `wind10m_speed` INT, 
  `pais` VARCHAR (20),
  PRIMARY KEY (`index`))
ENGINE = InnoDB;
'''

In [27]:
tabla2 = mi_bbdd.crear_insertar_tabla(tabla_clima)

In [28]:
for indice, fila in df_tiburon.iterrows(): # itreamos por el dataframe.
    
    
    query_tiburon = f"""
            INSERT INTO ataques (case_number, year, type, country, activity, injury, mes, deceso, genero, especies, edad) 
            VALUES ( "{fila['case_number']}", "{fila['year']}", "{fila['type']}", "{fila['country']}", "{fila['activity']}", "{fila['injury']}", "{fila['mes']}", "{fila['deceso']}", "{fila['genero']}", "{fila['especies']}", "{fila['edad']}");
            """
    mi_bbdd.crear_insertar_tabla(query_tiburon)
print("Ya estan los datos insertados en tu tabla")

Ya estan los datos insertados en tu tabla


In [29]:
for indice, fila in df_clima.iterrows(): # itreamos por el dataframe.
       
    query_clima = f"""
            INSERT INTO clima (timepoint, cloudcover, highcloud, midcloud, lowcloud, rh_profile, wind_profile, temp2m, lifted_index, rh2m, msl_pressure, prec_type, 
            prec_amount, snow_depth, wind10m_direction, wind10m_speed, pais) 
            VALUES ("{fila['timepoint']}", "{fila['cloudcover']}", "{fila['highcloud']}", "{fila['midcloud']}", "{fila['lowcloud']}", "{fila['rh_profile']}", "{fila['wind_profile']}", "{fila['temp2m']}", "{fila['lifted_index']}", "{fila['rh2m']}", 
            "{fila['msl_pressure']}", "{fila['prec_type']}", "{fila['prec_amount']}", "{fila['snow_depth']}", "{fila['wind10m.direction']}", "{fila['wind10m.speed']}", "{fila['pais']}");
            """
    mi_bbdd.crear_insertar_tabla(query_clima)
print("Ya estan los datos insertados en tu tabla")

Ya estan los datos insertados en tu tabla
